In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.driver.memory", '4g')
    .config("spark.executor.memory", '8g')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.parquet.enableVectorizedReader","false")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

22/09/15 14:28:52 WARN Utils: Your hostname, DESKTOP-5U9FK5P resolves to a loopback address: 127.0.1.1; using 172.26.185.11 instead (on interface eth0)
22/09/15 14:28:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/15 14:28:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.functions import *
from pyspark.sql import functions as F

### Load consumer data

In [3]:
sdf_consumer = spark.read.csv('../data/tables/tbl_consumer.csv', sep='|', header=True)
sdf_consumer.show(5, truncate=False)

+-----------------+-----------------------------+-----+--------+------+-----------+
|name             |address                      |state|postcode|gender|consumer_id|
+-----------------+-----------------------------+-----+--------+------+-----------+
|Yolanda Williams |413 Haney Gardens Apt. 742   |WA   |6935    |Female|1195503    |
|Mary Smith       |3764 Amber Oval              |NSW  |2782    |Female|179208     |
|Jill Jones MD    |40693 Henry Greens           |NT   |862     |Female|1194530    |
|Lindsay Jimenez  |00653 Davenport Crossroad    |NSW  |2780    |Female|154128     |
|Rebecca Blanchard|9271 Michael Manors Suite 651|WA   |6355    |Female|712975     |
+-----------------+-----------------------------+-----+--------+------+-----------+
only showing top 5 rows



In [4]:
sdf_consumer.count()

499999

### Load transactional data

In [5]:
# Reading in all transaction data and joining them
sdf_transactions1 = spark.read.parquet('../data/tables/transactions_20210228_20210827_snapshot')
sdf_transactions2 = spark.read.parquet('../data/tables/transactions_20210828_20220227_snapshot')
sdf_transactions3 = spark.read.parquet('../data/tables/transactions_20220228_20220828_snapshot')

sdf_transactions = sdf_transactions1.union(sdf_transactions2)
sdf_transactions = sdf_transactions.union(sdf_transactions3)

sdf_transactions.show(10, truncate=False)

+-------+------------+------------------+------------------------------------+--------------+
|user_id|merchant_abn|dollar_value      |order_id                            |order_datetime|
+-------+------------+------------------+------------------------------------+--------------+
|18478  |62191208634 |63.255848959735246|949a63c8-29f7-4ab0-ada4-99ac50a88952|2021-08-20    |
|2      |15549624934 |130.3505283105634 |6a84c3cf-612a-4574-835b-144a47353eff|2021-08-20    |
|18479  |64403598239 |120.15860593212783|b10dcc33-e53f-4254-863c-de5266810cbc|2021-08-20    |
|3      |60956456424 |136.6785200286976 |0f09c5a5-784e-4477-b049-8ee4dd069b7b|2021-08-20    |
|18479  |94493496784 |72.96316578355305 |f6c78c1a-4600-4c5f-8e97-6e9eb534b586|2021-08-20    |
|3      |76819856970 |448.529684285612  |5ace6a24-cdf0-4aa3-b571-1d9406b352b5|2021-08-20    |
|18479  |67609108741 |86.4040605836911  |d0e180f0-cb06-42a3-bd1a-c47dca15bc55|2021-08-20    |
|3      |34096466752 |301.5793450525113 |6fb1ff48-24bb-4f97-

In [6]:
sdf_transactions.count()

14195505

### Read consumer details (joining table)

In [7]:
sdf_userdetails = spark.read.parquet('../data/tables/consumer_user_details.parquet')
sdf_userdetails.show(10, truncate=False)

+-------+-----------+
|user_id|consumer_id|
+-------+-----------+
|1      |1195503    |
|2      |179208     |
|3      |1194530    |
|4      |154128     |
|5      |712975     |
|6      |407340     |
|7      |511685     |
|8      |448088     |
|9      |650435     |
|10     |1058499    |
+-------+-----------+
only showing top 10 rows



In [8]:
sdf_userdetails.count()

499999

### Join transactional data with consumer data

In [9]:
sdf_usertransaction = sdf_userdetails.join(sdf_transactions, on='user_id')
sdf_consumer_transaction = sdf_usertransaction.join(sdf_consumer, on='consumer_id')
sdf_consumer_transaction.show(5)

+-----------+-------+------------+------------------+--------------------+--------------+---------+--------------------+-----+--------+------+
|consumer_id|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|     name|             address|state|postcode|gender|
+-----------+-------+------------+------------------+--------------------+--------------+---------+--------------------+-----+--------+------+
|        551|   3471| 68216911708|  39.2325218595659|c921263e-489e-45a...|    2021-08-21|Todd Long|883 Patty Mountai...|  NSW|    2021|  Male|
|        551|   3471| 77698107389| 90.16967584866273|aaea14f5-48e8-4ef...|    2021-08-21|Todd Long|883 Patty Mountai...|  NSW|    2021|  Male|
|        551|   3471| 46451548968|3.8405569898888428|7e8e7fca-04a1-4f2...|    2021-08-19|Todd Long|883 Patty Mountai...|  NSW|    2021|  Male|
|        551|   3471| 35556933338| 66.18325079406301|13b0be6a-20b7-439...|    2021-08-22|Todd Long|883 Patty Mountai...|  NSW|    2021|  Male|

In [10]:
# Only keeping useful features
sdf_consumer_transaction = sdf_consumer_transaction.select("consumer_id","user_id",'merchant_abn','dollar_value',
                            'order_datetime','state','postcode','gender')
sdf_consumer_transaction.show(5)                   

+-----------+-------+------------+------------------+--------------+-----+--------+------+
|consumer_id|user_id|merchant_abn|      dollar_value|order_datetime|state|postcode|gender|
+-----------+-------+------------+------------------+--------------+-----+--------+------+
|        551|   3471| 68216911708|  39.2325218595659|    2021-08-21|  NSW|    2021|  Male|
|        551|   3471| 77698107389| 90.16967584866273|    2021-08-21|  NSW|    2021|  Male|
|        551|   3471| 46451548968|3.8405569898888428|    2021-08-19|  NSW|    2021|  Male|
|        551|   3471| 35556933338| 66.18325079406301|    2021-08-22|  NSW|    2021|  Male|
|        551|   3471| 91772719127|33.692747732339996|    2021-08-14|  NSW|    2021|  Male|
+-----------+-------+------------+------------------+--------------+-----+--------+------+
only showing top 5 rows



In [11]:
# Checking final count to see if it joined properly
sdf_consumer_transaction.count()

14195505

In [12]:
# Looking at descriptive statistics for outlier analysis
sdf_consumer_transaction.describe()

summary,consumer_id,user_id,merchant_abn,dollar_value,state,postcode,gender
count,14195505,14195505,14195505,14195505,14195505,14195505,14195505
mean,753392.9631019115,12040.444758957148,5.542435612115468...,166.22895390891804,null,4043.465215855301,null
stddev,435520.71235532145,6951.450003956091,2.532810330437655...,517.850560261282,null,1780.846490376443,null
min,30,1,10023283211,9.756658099412162E-8,ACT,1001,Female
max,1499867,24081,99990536339,105193.88578925544,WA,9999,Undisclosed


In [12]:
sdf_consumer_transaction.printSchema()

root
 |-- consumer_id: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- gender: string (nullable = true)



### Look at gender

In [13]:
# Checking gender categories
sdf_consumer_transaction.groupBy('gender').count()

gender,count
Undisclosed,1442457
Female,6308459
Male,6444589


With our genders, it may be a promising feature which woucl dbe used further along the line. Thus, we want to keep it for future references. However, there are people who wish to not disclose their gender and a gender feature as 'undisclosed' has appeared. The following code deals with attempting to find if there has been any past that customers who have said 'undisclosed' have provided us.

For the first set of data, doesn't seem like there's any overlap. Smoge useless analysis

In [14]:
# Checking if there is only correct states
sdf_consumer_transaction.groupBy('state').count()

state,count
NT,202178
ACT,130325
SA,1612955
TAS,525947
WA,2247663
QLD,2100381
VIC,3280823
NSW,4095233


### Round dollar values to 2dp

In [15]:
# Round to 2 decimal places, and define a transaction range
min_value = 0
max_value = 10000
sdf_consumer_transaction = sdf_consumer_transaction.withColumn('dollar_value', F.round(F.col('dollar_value'), 2))
sdf_consumer_transaction = sdf_consumer_transaction.where(
    (F.col('dollar_value') > min_value)
    & (F.col('dollar_value') <= max_value)
)
sdf_consumer_transaction.count()

14189700

### Check ABN validity

In [16]:
# Make sure ABN is valid, takes in long

def validateABN(merchant_abn):

    str_abn = str(merchant_abn)

    if len(str_abn) == 11:
        return True
    else:
        return False

In [17]:
# Create a list of all row values, used for validating ABN

sdf_list = sdf_consumer_transaction.select("merchant_abn").collect()

In [18]:
# Find any merchants without a valid ABN

i = 0
invalidABN = []

while i < len(sdf_list):
    abn = str(sdf_list[i].__getitem__('merchant_abn'))
    if validateABN(abn) == False:
        invalidABN.append(abn)
    i += 1

In [12]:
invalidABN

[]

ez no invalid abn

### Check for valid postcodes in Australia

In [19]:
# Make sure post codes in right range and only digits
min_postcode = '0200'
max_postcode = '9999'

sdf_consumer_transaction = sdf_consumer_transaction.where(
    (F.col('postcode') >= min_postcode)
    & (F.col('postcode') <= max_postcode)
)
sdf_consumer_transaction.count()

14189700

In [ ]:
# Check if postcode matches up with state


In [20]:
# Checking date range
start_date = '2021-02-28'
end_date = '2022-08-28'
sdf_consumer_transaction = sdf_consumer_transaction.where(
    (F.col('order_datetime') >= start_date) & (F.col('order_datetime') <= end_date)
)
sdf_consumer_transaction.count()

12529151

In [21]:
# Deal with NULL values
sdf_consumer_transaction = sdf_consumer_transaction.dropna('any')
sdf_consumer_transaction.count()

12529151

In [ ]:
# Checking descriptive statistics again
sdf_consumer_transaction.describe()

In [ ]:
# Export cleaned data